In [ ]:
!pip install sklearn
#!pip install opentsne
!pip install yellowbrick

# **INIT**

In [ ]:
import datetime
import pandas as pd
filename = './oil.csv'
print(f'importing file: {filename}')
time_of_script_run = datetime.datetime.today()
df = pd.read_csv(filename)
print(f'total rows in dataset: {len(df)}')
max_date_in_dataset = max(pd.to_datetime(df['date']))
min_date_in_dataset = min(pd.to_datetime(df['date']))
print(f'max date in dataset: {max_date_in_dataset}')
print(f'min date in dataset: {min_date_in_dataset}')

def is_exclude_current_month_year_data():
  if time_of_script_run.year == max_date_in_dataset.year and time_of_script_run.month == max_date_in_dataset.month:
    return True
  return False

def check_ignore_fluctuating_row(dataset,columns):
  input_to_predict = dataset[columns][:1]
  
  assert len(input_to_predict) == 1

  if is_exclude_current_month_year_data():
    print(f'dataset\'s first row is fluctuating data: {max_date_in_dataset.month}/{max_date_in_dataset.year}')
    print(f'***cannot train models on this row as prices are still fluctuating') 
    print(f'also not including the last row as it is an error row')   
    #print(f'length dataset:\n{len(dataset)-2}')
    dataset_with_no_fluctuating_row = dataset[1:-1]
    assert len(dataset) - len(dataset_with_no_fluctuating_row) == 2
    print(f'\ndataset with fluctuating data(first row) and last row:\n {dataset}')
    print(f'\ndataset w/o fluctuating data(first row) and last row:\n {dataset_with_no_fluctuating_row}')
    return dataset_with_no_fluctuating_row,input_to_predict
  
  return dataset,input_to_predict

# **RUN-THRU-CLUSTERS-NaN-PREDICTIONS**

In [ ]:
# df_lenght = len(df)

# for row_index in range(1,df_lenght,1):
#   print(f'row_index: {row_index}')

# filenames = ['3-opens-1-close.csv']
# clustering_columns = [
#   'zs_open',
#   'zm_open',
#   'zl_open',]  
# clustering_target = [
#   #'zs_close',]
#   #'zm_close',]
#   'zl_close'] 

In [2]:
import pandas as pd

filenames = [
  './zs-zm-zl.csv']
  #'./zm.csv',]
  #'./zl.csv',]
  #'./zs.csv']
# clustering_columns = [
#   'open_to_prev_open_diff',
#   'open_to_prev_high_diff',
#   'open_to_prev_low_diff',
#   'open_to_prev_close_diff',]  
# clustering_target = ['current_close_minus_current_open']

clustering_columns = [
  'zs_open_to_prev_open_diff',
  'zs_open_to_prev_high_diff',
  'zs_open_to_prev_low_diff',
  'zs_open_to_prev_close_diff',
  'zm_open_to_prev_open_diff',
  'zm_open_to_prev_high_diff',
  'zm_open_to_prev_low_diff',
  'zm_open_to_prev_close_diff',
  'zl_open_to_prev_open_diff',
  'zl_open_to_prev_high_diff',
  'zl_open_to_prev_low_diff',
  'zl_open_to_prev_close_diff',]  
clustering_targets = [
  ['zs_current_close_minus_current_open'],
  ['zs_current_low_minus_current_open'],
  ['zs_current_high_minus_current_open']]

for clustering_target in clustering_targets:
  for f in filenames:
    print(f'\n *************************************************************************************************************************************************** filename {f} ***************************************************************************************************************** \n')
    # m = datetime.now().month
    # datasets = [
    #   pd.read_csv(f),
    #   df[df['date'].str.contains(f'{m}/1/')]]
    
    #for df in datasets:
    #print(f'\n\n\n\n\n\n\n\n*********************************************************************************************************working with this data *********************************************************************************************************:\n{df.head()}')
    df = pd.read_csv(f)
    
    # user_input = input('run monthly? ')
    # if user_input == 'y':
    #   from datetime import datetime
    #   m = datetime.now().month
    #   df = df[df['date'].str.contains(f'{m}/1/')]

    start_data_at_1999 = input('start data at 1999?')
    if start_data_at_1999 == 'y':  
      df['date']= pd.to_datetime(df['date'])
      #print(df['date'])  
      df = df.loc[df['date'] >= '1999-01-01']
      #print(df['date'])
      #df = df[(df['date'] > '1999-01-01')]
      #df = df.query('date > 1999-01-01')
      min_df = min(df['date'])
      max_df = max(df['date'])
      print(f'using data from 1999 to now, min date in df: {min_df}, max date in df: {max_df}')    
      
    
    from pandas import DataFrame
    import matplotlib.pyplot as plt
    from sklearn.cluster import KMeans
    from sklearn import linear_model
    from sklearn import svm
    import pandas as pd
    from sklearn import linear_model
    from sklearn.ensemble import AdaBoostRegressor
    from sklearn.ensemble import BaggingRegressor
    from sklearn.svm import SVR
    from sklearn.ensemble import ExtraTreesRegressor
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.ensemble import RandomForestRegressor

    from sklearn.linear_model import RidgeCV
    from sklearn.svm import LinearSVR
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.ensemble import StackingRegressor
    from numpy import ravel
    #from textwrap import TextWrapper

    clustering_data_columns = df[clustering_columns + clustering_target][:-1]
    num_of_clusters = range(2,190,2)
    nan_predictions = []

    print(f'\n NaN Column: {clustering_target[0]}')

    print(f'\n clustering data columns: \n{clustering_data_columns.values.round(0)}\n')

    for num_of_cluster in num_of_clusters:
      try:
        print(f'\n\n\n########################################### running prediction for NaN column "{clustering_target[0]}" with {num_of_cluster} clusters')  
        training_dataset = clustering_data_columns[clustering_columns + clustering_target]   

        clusts = KMeans(
            n_clusters=num_of_cluster,
            algorithm='elkan',
            random_state=10)\
            .fit_predict(
                training_dataset[clustering_columns])
            
        print(f'\ncluster number is "{clusts[0]}" for input row {training_dataset[clustering_columns].values[0].round(1)}')
        clusts_df = pd.DataFrame(clusts,columns=['cluster_num'])
        joined_dataset = training_dataset.join(clusts_df)   

        cluster_rows = joined_dataset\
          .query(f'cluster_num == { clusts[0] }')
        
        column_names = '\n'.join(cluster_rows.columns.values)      
        print(f'NaN column: "{clustering_target[0]}"\n clustering rows: \n{ cluster_rows[clustering_columns+clustering_target].round(0).values } \n number of rows in cluster: {len(cluster_rows)}')      
        assert len(training_dataset) == len(clustering_data_columns[clustering_target])    

        fit_index = 1
        cluster_regression_training_dataset = cluster_rows[clustering_columns][fit_index:]
        cluster_regression_training_target = cluster_rows[clustering_target][fit_index:]
        cluster_regression_input_to_predict = cluster_rows[clustering_columns][:fit_index]

        assert len(cluster_regression_input_to_predict) == 1
        
        def is_class_balanced():
          is_clustering_target_greater_than_zero = cluster_rows[clustering_target][1:].ge(0) #.pct_change().ge(0)    
          _ravel_data = is_clustering_target_greater_than_zero[clustering_target].values.ravel()
          if True in _ravel_data and False in _ravel_data:
            #print(f'\n is greater than or equal to zero: {_ravel_data}')
            return True
          else:
            return False

        def train_get_model(n_splits):
        
          from sklearn.svm import LinearSVR
          from sklearn.gaussian_process import GaussianProcessRegressor
          from sklearn.isotonic import IsotonicRegression
          from sklearn.kernel_ridge import KernelRidge
          from sklearn.linear_model import RidgeCV        
          from sklearn.linear_model import Lars
          from sklearn.linear_model import ARDRegression
          from sklearn.linear_model import HuberRegressor
          from sklearn.linear_model import PassiveAggressiveRegressor
          from sklearn.linear_model import LassoLarsCV
          from sklearn.neighbors import KNeighborsRegressor
          from sklearn.neural_network import MLPRegressor
          from sklearn.tree import ExtraTreeRegressor
          estimators = [
              #('linear-lr', RidgeCV()),
              ('svr', LinearSVR(random_state=42,max_iter=100000)),
              ('gaussian', GaussianProcessRegressor(random_state=42)),                 
              ('kernel-ridge', KernelRidge()),
              ('neightbors', KNeighborsRegressor(n_neighbors=1)),
              ('neural', MLPRegressor(max_iter=8000,random_state=42)),
              ('tree', ExtraTreeRegressor(random_state=42)),                
          ]

          cluster_regression_model = StackingRegressor(    
              cv = n_splits,        
              estimators=estimators,
              final_estimator=RandomForestRegressor(
                  n_estimators=10000,                            
                  random_state=42))

          cluster_regression_model.fit(
              cluster_regression_training_dataset,
              cluster_regression_training_target.values.ravel())
          
          return cluster_regression_model

        def check_train_model_with_value_error():    
          try:
            trained_model = train_get_model(5)
            return trained_model
          except ValueError as training_exception:
            print(f'\n training exception: {training_exception}')
            print(f'\n decreasing n_splits and retraining the model')
            trained_model = train_get_model(2) #int(len(cluster_regression_training_dataset)/2)          
            return trained_model
          # finally:
            # return trained_model
            
        value_corrected_model = check_train_model_with_value_error()

        cluster_regression_prediction = value_corrected_model.predict(cluster_regression_input_to_predict)
        print(f'\n nan value prediction: {cluster_regression_prediction.round(0)}\n')  
        nan_predictions.append([num_of_cluster,cluster_regression_prediction[0].round(0),len(cluster_rows)])      

        if is_class_balanced() == False:
          print(f'class is not balanced for cluster {clusts[0]}, need negative and positive values in target column, stoping right here as more clusters will not change the imbalanced outcome')
          print('not breaking, break is commented out')
          #break
      except Exception as e:
        print(f'\n error occured {e}')
        #exception_type = type(e).__name__
        #print(f'{exception_type}')
        break

    final_pd = pd.DataFrame(
        nan_predictions,
        columns=[
          'num_of_clusters',
          'nan_prediction',
          'number_of_rows_in_cluster'])\
        .drop_duplicates()
    average_nan_prediction = final_pd['nan_prediction']\
        .mean()
        #.drop_duplicates()\

    max_nan_prediction = final_pd['nan_prediction']\
      .max()
    
    min_nan_prediction = final_pd['nan_prediction']\
      .min()
        

    print(f'\n cluster number/NaN prediction: \n{final_pd}\n\n')  
    print(f'NaN column "{clustering_target[0]}", average NaN prediction for file "{f}" is: {average_nan_prediction}\n\n')
    print(f'min NaN pred: {min_nan_prediction}, max NaN pred: {max_nan_prediction}')




 *************************************************************************************************************************************************** filename ./zs-zm-zl.csv ***************************************************************************************************************** 

start data at 1999?y
using data from 1999 to now, min date in df: 1999-01-01 00:00:00, max date in df: 2021-12-01 00:00:00

 NaN Column: zs_current_close_minus_current_open

 clustering data columns: 
[[-27. -67.  41. ...   1.   0.  nan]
 [ -6. -17.  64. ...   4.   0. -32.]
 [-39. -53.   6. ...   4.  -0.  -6.]
 ...
 [ 34. -21.  34. ...   2.  -0.  -8.]
 [-57. -68.   2. ...   0.   0.  33.]
 [-30. -52.   1. ...   0.   0. -58.]]




########################################### running prediction for NaN column "zs_current_close_minus_current_open" with 2 clusters

cluster number is "0" for input row [-27.  -67.2  40.8   4.8  10.  -33.9  14.    0.8  -5.8  -7.    0.8   0.3]
NaN column: "zs_current_close_minu


***CLUSTER ROWS SCATTER PLOTS***

In [ ]:
import seaborn as sns
sns.pairplot(
    data=cluster_rows,
    markers=["o","s","D","d","*"],
    aspect=2)  

# **CLUSTERING && CLUSTER REGRESSION**

In [ ]:
clustering_columns = [
  'open_to_prev_open_diff',
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]  
  #'open']
clustering_target = ['current_close_minus_current_open'] #['is_win'] #

clustering_data_columns = df[clustering_columns + clustering_target][:-1]
# clustering_dataset,clustering_input_to_predict = check_ignore_fluctuating_row(
#     clustering_data_columns,
#     clustering_columns)

from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import linear_model


num_of_cluster = 125
training_dataset = clustering_data_columns[clustering_columns+clustering_target]
print(f'\ncluster training dataset:\n{training_dataset.values}')
print(f'\ncluster training dataset description:\n{training_dataset.describe()}')

clusts = KMeans(
    n_clusters=num_of_cluster,
    algorithm='elkan',
    random_state=10)\
    .fit_predict(
        training_dataset[clustering_columns])
print(f'\ncluster number is "{clusts[0]}" for input row {training_dataset[clustering_columns].values[0]}')
print(f'\n clusters:\n{clusts} with length: {len(clusts)}\n')

clusts_df = pd.DataFrame(clusts,columns=['cluster_num'])
joined_dataset = training_dataset.join(clusts_df)
print(f'\n joined training dataset and cluster predictions:\n{ joined_dataset }')

print(f'\ncluster counts:\n{clusts_df.value_counts()}\n')

cluster_rows = joined_dataset\
  .query(f'cluster_num == { clusts[0] }')

column_names = '\n'.join(cluster_rows.columns.values)
print(f'\n"{len(cluster_rows)}" rows for cluster "{clusts[0]}"\n\n{column_names} \n\n{cluster_rows[clustering_columns+clustering_target].round(1).values}\n\n')

assert len(training_dataset) == len(clustering_data_columns[clustering_target])

# import seaborn as sns
# sns.pairplot(
#     data=joined_dataset,
#     hue='cluster_num',
#     #markers=["o","s","D","d","*"],
#     aspect=2.5)  

***CLUSTER REGRESSION***

In [ ]:
from sklearn import svm

#cluster_regression_model = linear_model.LinearRegression()
cluster_regression_model = svm.SVR()
fit_index = 1
cluster_regression_training_dataset = cluster_rows[clustering_columns][fit_index:]
cluster_regression_training_target = cluster_rows[clustering_target][fit_index:]
cluster_regression_input_to_predict = cluster_rows[clustering_columns][:fit_index]

assert len(cluster_regression_input_to_predict) == 1

print(f'cluster regression training/target dataset:\n{cluster_regression_training_dataset.join(cluster_regression_training_target).values}\n')
#print(f'cluster regression training target:\n{cluster_regression_training_target}\n')
cluster_regression_model.fit(
    cluster_regression_training_dataset,
    cluster_regression_training_target)
print(f'\n cluster regression input to predict column {clustering_target}:\n {cluster_regression_input_to_predict.values}')
cluster_regression_prediction = cluster_regression_model.predict(cluster_regression_input_to_predict)
print(f'\n nan value prediction: {cluster_regression_prediction.round(0)}\n')

# **DISTRIBUTIONS**

In [ ]:
import seaborn as sns

distribution_columns = [
  'open_to_prev_open_diff',
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]

other_columns = [
  'current_close_minus_current_open',	]
  #'is_current_close_minus_current_open_above_zero',]
  #'is_win',
  #'is_short_win',
  #'is_short_loss',	
  #'is_long_win',
  #'is_long_loss']  

# for column in distribution_columns:
#   ax = sns.displot(df[column][1:-1], kde=True)
#   ax.set(xlabel=column, ylabel='Frequency')

sns.pairplot(
    data=df[distribution_columns + other_columns],
    aspect=2)

# **ONE-CLASS**








In [ ]:
from sklearn.svm import OneClassSVM

one_class_columns = [
  'open_to_prev_open_diff',
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]
  #'open']
drop_columns = ['is_win']

one_class_svm_data_columns = df[one_class_columns + drop_columns]

one_class_dataset, input_to_predict = check_ignore_fluctuating_row(
    one_class_svm_data_columns,
    one_class_columns)

one_class_svm_training_data_wins = one_class_dataset.query('is_win == 1')[one_class_columns]
one_class_svm_training_data_losses = one_class_dataset.query('is_win == 0')[one_class_columns]
 
print(f'\n one class training data wins:\n{one_class_svm_training_data_wins}')
print(f'\n one class training data losses:\n{one_class_svm_training_data_losses}')

win_model = OneClassSVM(gamma='auto').fit(one_class_svm_training_data_wins)
loss_model = OneClassSVM(gamma='auto').fit(one_class_svm_training_data_losses)

is_winners = win_model.predict(one_class_svm_training_data_wins)
is_losers = loss_model.predict(one_class_svm_training_data_losses)
print(f'\nis_winning_onself {is_winners}\nis_loosing_onself {is_losers}\n')

is_winners_on_loosers_data = win_model.predict(one_class_svm_training_data_losses)
is_losers_on_winners_data = loss_model.predict(one_class_svm_training_data_wins)
print(f'\nis_winning_class_test_with_loosers {is_winners_on_loosers_data}\nis_loosing_class_test_with_winners {is_losers_on_winners_data}\n')

stage_2_one_class_svm_wins = [win for win in one_class_svm_training_data_wins.values if win_model.predict([win]) == -1]
print(f'found {len(stage_2_one_class_svm_wins)} winners for stage 2')
stage_2_win_model = OneClassSVM(gamma='auto').fit(stage_2_one_class_svm_wins)

stage_2_one_class_svm_losses = [loss for loss in one_class_svm_training_data_losses.values if loss_model.predict([loss]) == -1]
print(f'found {len(stage_2_one_class_svm_losses)} losers for stage 2')
stage_2_loss_model = OneClassSVM(gamma='auto').fit(stage_2_one_class_svm_losses)

print(f'\n one class inputs for prediction:\n{input_to_predict.values}')
is_winner_prediction = win_model.predict(input_to_predict)
is_loser_prediction = loss_model.predict(input_to_predict)
print(f'\nis_winner_stage_1_class {is_winner_prediction}\nis_looser_stage_1_class {is_loser_prediction}')
is_winner_stage_2_prediction = stage_2_win_model.predict(input_to_predict)
is_loser_stage_2_prediction = stage_2_loss_model.predict(input_to_predict)
print(f'is_winner_stage_2_class {is_winner_stage_2_prediction}\nis_loser_stage_2_class {is_loser_stage_2_prediction}')


#is_stage_2_winners = stage_2_win_model.predict(stage_2_one_class_svm_wins)
#print(f'is_stage_2_winners_onself:\n {is_stage_2_winners}')
#print(stage_2_win_model.predict(input_to_predict))
#is_stage_2_losers = stage_2_loss_model.predict(stage_2_one_class_svm_losses)
#print(f'is_stage_2_losers_onself:\n{is_stage_2_losers}')

In [ ]:
from sklearn.svm import OneClassSVM

crosshairs_columns = [
  'open_to_prev_open_diff',
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',	
  'open_to_prev_close_diff',]
  #'open',]
  #'high',
  #'low',
  #'close']

crosshairs_query_columns = [
  'is_short_win',
  'is_short_loss',
  'is_long_win',
  'is_long_loss']

crosshairs_data_columns = df[crosshairs_columns + crosshairs_query_columns]

crosshairs_dataset, crosshairs_input_to_predict = check_ignore_fluctuating_row(
    crosshairs_data_columns,
    crosshairs_columns)  

is_short_wins_one_class_dataset = crosshairs_dataset.query('is_short_win == 1')[crosshairs_columns]
is_short_losses_one_class_dataset = crosshairs_dataset.query('is_short_loss == 1')[crosshairs_columns]
is_long_wins_one_class_dataset = crosshairs_dataset.query('is_long_win == 1')[crosshairs_columns]
is_long_losses_one_class_dataset = crosshairs_dataset.query('is_long_loss == 1')[crosshairs_columns]
print(f'''\n
num of shorts wins: {len(is_short_wins_one_class_dataset)}
num of short losses: {len(is_short_losses_one_class_dataset)}
num of long wins: {len(is_long_wins_one_class_dataset)}
num of long losses: {len(is_long_losses_one_class_dataset)}''')

print(f'\nis_short_wins_one_class_dataset head:\n{is_short_wins_one_class_dataset.head()}')

short_wins_model = OneClassSVM(gamma='auto').fit(is_short_wins_one_class_dataset)
short_losses_model = OneClassSVM(gamma='auto').fit(is_short_losses_one_class_dataset)
long_wins_model = OneClassSVM(gamma='auto').fit(is_long_wins_one_class_dataset)
long_losses_model = OneClassSVM(gamma='auto').fit(is_long_losses_one_class_dataset)

short_wins_onself = short_wins_model.predict(is_short_wins_one_class_dataset)
short_losses_onself = short_losses_model.predict(is_short_losses_one_class_dataset)
long_wins_onself = long_wins_model.predict(is_long_wins_one_class_dataset)
long_shorts_onself = long_losses_model.predict(is_long_losses_one_class_dataset)

print(f'''\n
short wins onself: {short_wins_onself}
short losses onself: {short_losses_onself}
long wins onself: {long_wins_onself}
long shorts onself: {long_shorts_onself}
''')

print(f'crosshairs input to predict on:\n{crosshairs_input_to_predict.values}')

is_short_win_prediction = short_wins_model.predict(crosshairs_input_to_predict)
is_short_loss_prediction = short_losses_model.predict(crosshairs_input_to_predict)
is_long_win_prediction = long_wins_model.predict(crosshairs_input_to_predict)
is_long_loss_prediction = long_losses_model.predict(crosshairs_input_to_predict)

print(f'''\n
is_short_win_class {is_short_win_prediction}
is_short_loss_class {is_short_loss_prediction}
is_long_win_class {is_long_win_prediction}
is_long_loss_class {is_long_loss_prediction}
''')

# **REGRESSION**

In [ ]:
import pandas as pd
from sklearn import linear_model
from sklearn import svm

regression_columns = [
  'open_to_prev_open_diff',
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]
  #'open']
regression_target = ['current_close_minus_current_open']

regression_data_columns = df[regression_columns + regression_target]
regression_dataset, regression_input_to_predict = check_ignore_fluctuating_row(
    regression_data_columns,
    regression_columns)

assert len(regression_dataset[regression_columns]) == len(regression_dataset[regression_target])

#regression_model = linear_model.LinearRegression()
regression_model = svm.SVR()
regression_model.fit(
    regression_dataset[regression_columns],
    regression_dataset[regression_target])
print(f'regression input for prediction:\n {regression_input_to_predict.values}')
regression_prediction = regression_model.predict(regression_input_to_predict)
print(regression_prediction)



In [ ]:
scatter_x_column = ['open']
scatter_y_column = ['current_close_minus_current_open']

count_times_above_zero = regression_dataset[scatter_y_column].query(f'{scatter_y_column[0]} > 0')
count_times_below_zero = regression_dataset[scatter_y_column].query(f'{scatter_y_column[0]} < 0')

print(f'column: "{scatter_y_column[0]}"":\n cluster values above zero: {len(count_times_above_zero)}\n cluster values below zero: {len(count_times_below_zero)}')

import seaborn as sns
sns.pairplot(data=regression_dataset)

# **CLASSIFICATION**

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit

def grid_search(x,y):
  C_range = range(1,1000000,10000) 
  gamma_range = range(1,1000000,10000) 
  print(f'\n C_range: {C_range}\n\ngamma_range: {gamma_range}')
  param_grid = dict(gamma=gamma_range, C=C_range)
  cv = StratifiedShuffleSplit(
      n_splits=5, 
      test_size=0.2, 
      random_state=42)

  grid = GridSearchCV(
      SVC(), 
      param_grid=param_grid, 
      cv=cv)

  print(f'\n y_training_data:\n{y_training_data}')
  grid.fit(
      x, 
      y.values.ravel())
  print("\n The best parameters are %s with a score of %0.2f"
        % (grid.best_params_, grid.best_score_))
  return grid

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

targets = [
  ['is_current_close_minus_current_open_above_zero'],
  ['is_win'],
  ['is_short_win'],	
  ['is_short_loss'],	
  ['is_long_win'],	
  ['is_long_loss']]


#classification_target = ['is_win']
for classification_target in targets:
  print(f'\n\n\n\n#################################################### running classification for target: {classification_target} ####################################################')

  classification_columns = [
    'open_to_prev_open_diff',
    'open_to_prev_high_diff',
    'open_to_prev_low_diff',
    'open_to_prev_close_diff',]
    #'open']     

  classification_data_columns = df[classification_columns + classification_target]
  
  classification_dataset, classification_input_to_predict = check_ignore_fluctuating_row(
      classification_data_columns,
      classification_columns)

  x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(
      classification_dataset[classification_columns], 
      classification_dataset[classification_target], 
      test_size = 0.05)
  
  # grid = grid_search(
  #     x_training_data,
  #     y_training_data)

  classification_model = SVC(  
      random_state=100,    
      kernel='rbf', 
      C=10000,
      #gamma=0.1,
      #gamma='auto',
      #C = int(grid.best_params_['C']),
      #gamma = int(grid.best_params_['gamma']),     
      class_weight='balanced')
  
  classification_model.fit(
      x_training_data, 
      y_training_data.values.ravel())
  
  classification_performance = classification_model.predict(x_test_data) #x_test_data
  print(f'\nclassification performance on self:\n {classification_performance}')
  print('\n')
  print(classification_report(y_test_data, classification_performance))
  print(confusion_matrix(y_test_data, classification_performance))
  print('\n')
  print(f'classification input to predict:\n {classification_input_to_predict.values}')
  classification_prediction = classification_model.predict(classification_input_to_predict)
  print(f'\nclassification {classification_target} prediction:\n{classification_prediction}')

# **LAGS && AUTO-CORRELATION**

In [ ]:
import pandas as pd

lag = 20

lag_columns = [
  'current_close_minus_current_open',]
  #'open_to_prev_open_diff',]
  #'open_to_prev_high_diff',]
  #'open_to_prev_low_diff',]
  #'open_to_prev_close_diff',]

data = df[lag_columns][1:-1]

#for lag_column in lag_columns:
print(f'lag plot input data:\n{df[lag_columns]}\n')
pd.plotting.lag_plot(data, lag = lag)


from statsmodels.graphics import tsaplots
import matplotlib.pyplot as plt
fig = tsaplots.plot_acf(data, lags=lag)
plt.show()

import statsmodels.api as sm
sm.tsa.acf(data).round(3)

# **FEATURE IMPORTANCE**

In [ ]:

from matplotlib import pyplot
importance = regression_model.coef_
print(f'\n feature importance:\n{importance}')

# **TSNE**

In [ ]:
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns

tsne_columns = [
  'open_to_prev_open_diff',	
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]
  #'current_close_minus_current_open']

tsne_target = ['is_win']

tsne_training_data = df[tsne_columns + tsne_target][1:-1]
 
tsne = TSNE(
  n_components=2,
  #perplexity=100,
  #learning_rate=10,
  n_iter=1000000,)
  #random_state=1,
  #init='pca')

tsne_result = tsne.fit_transform(tsne_training_data)
print(f'\n tSNE shape: {tsne_result.shape}\n tSNE result: {tsne_result}')

#import seaborn as sns
#sns.pairplot(data=pd.DataFrame(tsne_result))

#pd_tsne_result = pd.DataFrame(tsne_result,columns=['x','y','z'])
pd_tsne_result = pd.DataFrame(tsne_result,columns=['x','y'])

print(f'pandas tsne_results:\n{pd_tsne_result}')

import seaborn as sns
sns.pairplot(
    data=pd_tsne_result,
    #hue='cluster_num',
    #markers=["o","s","D","d","*"],
    aspect=2.5)  


import re, seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

fig = plt.figure(figsize=(15,15))
ax = Axes3D(fig)
fig.add_axes(ax)

# get colormap from seaborn
cmap = ListedColormap(sns.color_palette("husl", 256).as_hex())

# plot
#sc = ax.scatter(x, y, z, s=40, c=x, marker='o', cmap=cmap, alpha=1)
sc = ax.scatter(
  pd_tsne_result['x'], 
  pd_tsne_result['y'], 
  pd_tsne_result['z'])

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

pd_tsne_result.to_csv("tsne-plotly-chart-studio.csv")


In [ ]:
#from openTSNE import TSNE as openTSNE

tsne_columns = [
  'open_to_prev_open_diff',	
  'open_to_prev_high_diff',
  'open_to_prev_low_diff',
  'open_to_prev_close_diff',]
  #'current_close_minus_current_open']

tsne_target = ['is_win']

X = df[tsne_columns][1:-1]
y = df[tsne_target][1:-1]

print(f'x head: {X}\n y head: {y}')

from yellowbrick.text.tsne import tsne
from sklearn.feature_extraction.text import TfidfVectorizer
from yellowbrick.datasets import load_hobbies

Load the data and create document vectors
corpus = load_hobbies()
tfidf = TfidfVectorizer()

X = tfidf.fit_transform(corpus.data)
y = corpus.target

tsne(X, y)

open_tsne = TSNE(
  n_components=2,
  n_iter=10000)

print(df[tsne_columns])

open_tsne_result = openTSNE.fit(df[tsne_columns])

print(f'open tsne results:\n{open_tsne_result}')
